In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import numpy as np
import random
import sys
sys.path.append('../tools')


from sklearn.model_selection import train_test_split
from models.kaggle_dog_dataloader import *
from active_learning_utility import get_strategy
from active_learning_data import Data
from active_learning_net import Net
from models.kaggle_dog_dataloader import ActiveDataHandler
from copy import deepcopy
from models.SwaV import SwaV
from models.SupervisedDownstream import SupervisedDownstream
import warnings

warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argument*")
warnings.filterwarnings("ignore", ".*Set a lower value for log_every_n_steps if you want to see logs for the training epoch*")
warnings.filterwarnings("ignore", ".*exists and is not empty*")
warnings.filterwarnings("ignore", ".*Checkpoint directory {dirpath} exists and is not empty*")


In [3]:
data_dir = "../../../user_data/competition_data/clips"
log_folder_root = '../../../user_data/logs/'
ckpt_folder_root = '../../../user_data/checkpoints/'
strategy_name = 'WAAL'

targets = [
    'Dog_1',
    'Dog_2',
    'Dog_3',
    'Dog_4',
]

# set the pipeline to be deterministic
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
determine_generator = torch.Generator()
determine_generator.manual_seed(random_seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    # True ensures the algorithm selected by CUFA is deterministic
    torch.backends.cudnn.deterministic = True
    # torch.set_deterministic(True)
    # False ensures CUDA select the same algorithm each time the application is run
    torch.backends.cudnn.benchmark = False

In [4]:
nStart = 3.7
nEnd = 15
nQuery = 1

In [5]:
args_task = {'n_epoch': 100,
             'transform_train': True,
             'strategy_name': strategy_name,
             'transform': False,
             'loader_tr_args': {'batch_size': 256, 'num_workers': 0, 'collate_fn': collate_fn,
                                'drop_last': True},
             'loader_te_args': {'batch_size': 256, 'num_workers': 2, 'collate_fn': collate_fn,
                                'drop_last': True}
             }

In [6]:
data, label = load_annotated_data(data_dir, targets)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.30, random_state=random_seed)

n_pool = len(y_train)
n_test = len(y_test)
NUM_INIT_LB = int(nStart * n_pool / 100)
NUM_QUERY = int(nQuery * n_pool / 100) if nStart != 100 else 0
NUM_ROUND = int((int(nEnd * n_pool / 100) - NUM_INIT_LB) / NUM_QUERY) if nStart != 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd * n_pool / 100) - NUM_INIT_LB) % NUM_QUERY != 0:
        NUM_ROUND += 1

print(NUM_INIT_LB)
print(NUM_QUERY)
print(NUM_ROUND)

Loading data
(0s)
X (178, 16, 400) y (178,) latencies (178,)
Loading data
(0s)
X (172, 16, 400) y (172,) latencies (172,)
Loading data
(1s)
X (480, 16, 400) y (480,) latencies (480,)
Loading data
(1s)
X (257, 16, 400) y (257,) latencies (257,)
Loading data
(1s)
X (418, 16, 400) y (418,)
Loading data
(4s)
X (1148, 16, 400) y (1148,)
Loading data
(19s)
X (4760, 16, 400) y (4760,)
Loading data
(11s)
X (2790, 16, 400) y (2790,)
(10203, 16, 400)
264
71
12


In [7]:
dataset = Data(X_train, y_train, X_test, y_test, ActiveDataHandler, args_task)

In [8]:
swav = SwaV().load_from_checkpoint(
    ckpt_folder_root + 'kaggle_dog_swav_34/kaggle_dog_swav-epoch=116-swav_loss=2.73583.ckpt')
model = SupervisedDownstream(swav.backbone)
# initialize model and save the model state
modelstate = deepcopy(model.state_dict())
device = "cuda" if torch.cuda.is_available() else "cpu"
net = Net(model, args_task, device)

In [ ]:
from models.WAAL_net import *

In [ ]:
clf = Classifier(swav.backbone)

In [9]:
strategy = get_strategy(strategy_name, dataset, net, None, args_task)

In [10]:
# initial round of training, round 0
dataset.initialize_labels(NUM_INIT_LB)
strategy.train()

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[19, 37, 50, 65, 79, 101, 132, 177, 185, 230, 233, 239, 245, 251, 263, 308, 318, 321, 332, 381, 393, 408, 435, 457, 468, 472, 500, 503, 538, 544, 676, 683, 696, 733, 752, 763, 764, 852, 865, 957, 970, 982, 1009, 1020, 1025, 1055, 1075, 1097, 1163, 1175, 1226, 1297, 1298, 1315, 1317, 1352, 1375, 1414, 1465, 1487, 1562, 1582, 1634, 1740, 1755, 1760, 1768, 1835, 1839, 1876, 1921, 1926, 1934, 2018, 2069, 2098, 2110, 2115, 2117, 2173, 2210, 2221, 2239, 2266, 2275, 2303, 2333, 2354, 2375, 2407, 2516, 2540, 2549, 2561, 2629, 2651, 2688, 2705, 2706, 2778, 2815, 2829, 2835, 2846, 2855, 2864, 2876, 2877, 2925, 2948, 2954, 2984, 3002, 3006, 3014, 3034, 3039, 3091, 3101, 3107, 3151, 3193, 3197, 3213, 3238, 3248, 3270, 3295, 3305, 3353, 3366, 3458, 3492, 3513, 3541, 3593, 3607, 3616, 3617, 3641, 3684, 3757, 3768, 3784, 3820, 3842, 3912, 3915, 4041, 4054, 4056, 4058, 4084, 4093, 4113, 4144, 4314, 4359, 4397, 4446, 4447, 4462, 4507, 4510, 4520, 4535, 4558, 4603, 4623, 4634, 4635, 4707, 4901, 4918, 49

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [11]:
for rd in range(1, NUM_ROUND +1):
    print('round ' + str(rd))
    q_idxs = strategy.query(NUM_QUERY)
    strategy.update(q_idxs)
    strategy.net.round = rd
    strategy.net.net.load_state_dict(modelstate)
    strategy.train()

In [12]:
q_idxs = strategy.query(NUM_QUERY)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000002710CF208B0>
Traceback (most recent call last):
  File "C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1474, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\multiprocessing\popen_spawn_win32.py", line 108, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
KeyboardInterrupt: 

KeyboardInterrupt

